# Combining Python, SQL DBs

## Aim: Using python, create a DB on your local SQLite instanc, insert data into it, and query that data to answer questions. 


# Using Python with a SQL DB

## Creating DB

We are going to create a DB that will have many tables inside it. Using SQLite, it is very easy to create a new database.  We simply just tell python to connect to a DB and if it doesn't already exist, it will create it.  

In [195]:
import sqlite3

In [196]:
conn = sqlite3.connect('tutorial.db')
cursor = conn.cursor()

For other flavors of SQL you will have to explicity run a statement like `CREATE DATABASE database_name`  

In order to do this, we would first need to connect to the database. Below is example code showing how you would connect to a DB.  You would just need to change out the different paramaters with the specific os your DB  

In [197]:
## importing 'mysql.connector' 
import mysql.connector 

## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector .connect(
    host = "localhost", #Usually an IP address
    user = "root",
    passwd = "dbms"
)

ModuleNotFoundError: No module named 'mysql'

In [198]:
from mysql.connector import errorcode

ModuleNotFoundError: No module named 'mysql'

In [199]:
db_name = 'employees'

Function to help us create a database

In [200]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [201]:
# check to see if the database already 
try:
    cursor.execute("USE {}".format(db_name))

#if the previous line fails because there isn't a db by that name run this line

except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

NameError: name 'mysql' is not defined

### Creating tables

Below are a list of SQL statements that will create different tables in a database.  We are going to loop over these `CREATE` statements and create a bunch of different tables. 

In [202]:


TABLES = {}
TABLES['employees'] = (
    "CREATE TABLE employees ("
    "  emp_no int(11) NOT NULL AUTO_INCREMENT,"
    "  birth_date date NOT NULL,"
    "  first_name varchar(14) NOT NULL,"
    "  last_name varchar(16) NOT NULL,"
    "  gender enum('M','F') NOT NULL,"
    "  hire_date date NOT NULL,"
    "  PRIMARY KEY (emp_no)"
    ") ENGINE=InnoDB")

TABLES['departments'] = (
    "CREATE TABLE departments ("
    "  dept_no char(4) NOT NULL,"
    "  dept_name varchar(40) NOT NULL,"
    "  PRIMARY KEY (dept_no), UNIQUE KEY dept_name (dept_name)"
    ") ENGINE=InnoDB")

TABLES['salaries'] = (
    "CREATE TABLE salaries ("
    "  emp_no int(11) NOT NULL,"
    "  salary int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT salaries_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_emp'] = (
    "CREATE TABLE dept_emp ("
    "  emp_no int(11) NOT NULL,"
    "  dept_no char(4) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no), KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_emp_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_emp_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_manager'] = (
    "  CREATE TABLE dept_manager ("
    "  dept_no char(4) NOT NULL,"
    "  emp_no int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no),"
    "  KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_manager_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_manager_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['titles'] = (
    "CREATE TABLE titles ("
    "  emp_no int(11) NOT NULL,"
    "  title varchar(50) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date DEFAULT NULL,"
    "  PRIMARY KEY (emp_no,title,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT titles_ibfk_1 FOREIGN KEY (emp_no)"
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

In [203]:
TABLES['employees']

"CREATE TABLE employees (  emp_no int(11) NOT NULL AUTO_INCREMENT,  birth_date date NOT NULL,  first_name varchar(14) NOT NULL,  last_name varchar(16) NOT NULL,  gender enum('M','F') NOT NULL,  hire_date date NOT NULL,  PRIMARY KEY (emp_no)) ENGINE=InnoDB"

In [204]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

#commit the changes you've made
cnx.commit()

Creating table employees: 

NameError: name 'mysql' is not defined

You don't need to do these two steps if you plan on continuing to do more work on the DB

In [205]:
#close your cursor
cursor.close()
#close the connection

cnx.close()

NameError: name 'cnx' is not defined

## Inserting Data

In [206]:
from datetime import date, datetime, timedelta
#package from python called datetime

In [207]:
tomorrow = datetime.now().date() + timedelta(days=1)

In [208]:
# insert statement to add an employee
add_employee = ("INSERT INTO employees "
               "(first_name, last_name, hire_date, gender, birth_date) "
               "VALUES (%s, %s, %s, %s, %s)")
# %s so that the inputs can be dynamically made
# 

#data to insert for one employee
data_employee = ('Geert', 'Vanderkelen', tomorrow, 'M', date(1977, 6, 14))



In [209]:
# Insert new employee
cursor.execute(add_employee, data_employee)
# Make sure data is committed to the database
cnx.commit()

ProgrammingError: Cannot operate on a closed cursor.

In [210]:
# grabs the id of the employee we just inserted so we can use it to input salary data
emp_no = cursor.lastrowid

In [211]:
#insert statement to add a salary
add_salary = ("INSERT INTO salaries "
              "(emp_no, salary, from_date, to_date) "
              "VALUES (%(emp_no)s, %(salary)s, %(from_date)s, %(to_date)s)")
# %(emp_no)s combines with dictionary made below 

In [212]:

# create salary information
data_salary = {
  'emp_no': emp_no,
  'salary': 50000,
  'from_date': tomorrow,
  'to_date': date(9999, 1, 1),
}

In [213]:

#Insert that salary information
cursor.execute(add_salary, data_salary)

# Make sure data is committed to the database
cnx.commit()

ProgrammingError: Cannot operate on a closed cursor.

## Insert Many

If you have a list of records that you would like to input, there are two ways you can do this.  YOu can loop over each record in python and execute each record, or you can use the `executemany()` method.  

`cursor.executemany(operation, list_of_params)`



In [214]:
#create a list of records

data = [
  ('Jane', date(2005, 2, 12)),
  ('Joe', date(2006, 5, 23)),
  ('John', date(2010, 10, 3)),
]

In [215]:
#create the insert statment
stmt = """INSERT INTO employees 
(first_name, hire_date) 
VALUES (%s, %s)"""
# Do not have to tell in a tuple

In [216]:
#insert all of the records and commit it
cursor.executemany(stmt, data)
cnx.commit()

ProgrammingError: Cannot operate on a closed cursor.

In [217]:
#Close the connection 

cursor.close()
cnx.close()

NameError: name 'cnx' is not defined

## Quyerying the DB

In [218]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

NameError: name 'mysql' is not defined

In [219]:

query = ("SELECT first_name, last_name, hire_date FROM employees "
         "WHERE hire_date BETWEEN %s AND %s")

hire_start = datetime.date(2006, 1, 1)
hire_end = datetime.date(2006, 12, 31)

cursor.execute(query, (hire_start, hire_end))



TypeError: descriptor 'date' for 'datetime.datetime' objects doesn't apply to a 'int' object

### How to read the data from your query.

You can use the `fetchone()`, `fetchmany()`, or `fetchall()` methods.
 


The `fetchone()` method returns the next row of a query result set or `None` in case there is no row left.

        row = cursor.fetchone()
 
        while row is not None:
            print(row)
            row = cursor.fetchone()

In case the number of rows in the table is small, you can use the `fetchall()` method to fetch all rows from the database table.

        rows = cursor.fetchall()
 
        print('Total Row(s):', cursor.rowcount)
        for row in rows:
            print(row)

**Why might you use a fetchone() when you have a query that will return multiple rows?**

For a relatively big table, it takes time to fetch all rows and return the entire result set. In addition, `fetchall()` needs to allocate enough memory to store the entire result set in the memory, which is not efficient.

MySQL Connector/Python has the `fetchmany()` method that returns the next number of rows (n) of the result set, which allows you to balance between retrieval time and memory space.



First, develop a generator that chunks the database calls into a series of `fetchmany()` calls:

In [220]:
def iter_row(cursor, size=10):
    while True:
        rows = cursor.fetchmany(size)
        if not rows:
            break
        for row in rows:
            yield row

Second, use the  iter_row() generator to fetch 10 rows at a time :

`cursor.execute("SELECT * FROM books")`
 
`for row in iter_row(cursor, 10):
    print(row)`

In [1]:
#in practice

data = cursor.fetchall()

NameError: name 'cursor' is not defined

In [2]:
print(data)

NameError: name 'data' is not defined

In [223]:
cursor.close()
cnx.close()

NameError: name 'cnx' is not defined

## Applied: Working with our student data

### Import our student data

In [1]:
import json
import pandas as pd
import sqlite3
import re
from datetime import date, datetime, timedelta


f=open('students_info.json','r')
students=json.load(f)


conn = sqlite3.connect('students.db')
cur = conn.cursor()

In [2]:
#examine that student data
students

[{'name': 'Sean Abu Wilson',
  'birthday': '02/06/1985',
  'fav_food': 'Bread',
  'birthplace': 'Birmingham, AL',
  'years_in_nyc': 9},
 {'name': 'christa',
  'birthday': '01/11/1996',
  'fav_food': 'buffalo wings',
  'birthplace': 'new york',
  'years_in_nyc': 2},
 {'name': 'Dariga Kokenova',
  'birthday': '04/19/1987',
  'fav_food': 'Italian',
  'birthplace': 'Kazakhstan',
  'years_in_nyc': 14},
 {'name': 'Rafael Ferreira',
  'birthday': '10/13/1991',
  'fav_food': 'Linguine con Vongole',
  'birthplace': 'Denver, CO',
  'years_in_nyc': 5},
 {'name': 'Saad Raees',
  'birthday': '05-13-1992',
  'fav_food': 'ice cream',
  'birthplace': 'Karachi,Pakistan',
  'years_in_nyc': 10},
 {'name': 'Ivan Zakharchuk',
  'birthday': '01.27.1988',
  'fav_food': 'Mediterranean',
  'birthplace': 'Kiev',
  'years_in_nyc': 10},
 {'name': 'Anjanette Vanessa Jia Craynock Santiago Delgado Betancourt Rivera',
  'birthday': '10-03-1987',
  'fav_food': ['Japanese', 'Burgers', 'Authentic Bacalaitos'],
  'birthp



### Create a connection and cursor to SQLite db

### Determine how you should define your table to hold the data

### Create a table for our student info

In [3]:

create_query = """ CREATE TABLE students
                    (name TEXT NOT NULL,
                    birthday TEXT NOT NULL,
                    fav_food TEXT NOT NULL,
                    birthplace TEXT NOT NULL,
                    years_in_nyc INTEGER default 0);
"""

In [4]:
cur.execute('DROP TABLE IF EXISTS students;')

In [5]:
cur.execute(create_query)

### Now that we have created our table we need to insert our data into the table.

We can't just pass the execute query a list of dictioanries. We need to parse into a list of tuples first. 

Take one student and identify how you would parse that information into a usable format.

Now use a for loop to loop over all of data and return a list that we can use for `executemany()`.

**If all of your data isn't formatted as expected, this will cause your loop to fail**  you might have to account for 'bad data' in your for loop.

Once you can parse the one students dictionary let's use a for loop to do that to all the students. 

When you try to parse it the first time, it is possible that the code could break because the data isn't in the format you expected.

In [14]:
def date_time(birthday):
    split_date = re.split('[/ -]|[.]', birthday)
    m,d,y = split_date
    d_o = date(int(y), int(m), int(d))
    
    return datetime.strftime(d_o , '%m/%d/%y')

In [15]:
# your code here
students_list = []
for student in students:
    if type(student['fav_food']) == list:
        for fav_food in student['fav_food']:
            student_tuple = (student['name'], 
                             date_time(student['birthday']),
                             fav_food,
                             student['birthplace'],
                             student['years_in_nyc'])
            students_list.append(student_tuple)
    else:
        student_tuple = (student['name'], 
                         date_time(student['birthday']),
                         student['fav_food'],
                         student['birthplace'],
                         student['years_in_nyc'])
        students_list.append(student_tuple)



In [16]:
students_list

[('Sean Abu Wilson', '02/06/85', 'Bread', 'Birmingham, AL', 9),
 ('christa', '01/11/96', 'buffalo wings', 'new york', 2),
 ('Dariga Kokenova', '04/19/87', 'Italian', 'Kazakhstan', 14),
 ('Rafael Ferreira', '10/13/91', 'Linguine con Vongole', 'Denver, CO', 5),
 ('Saad Raees', '05/13/92', 'ice cream', 'Karachi,Pakistan', 10),
 ('Ivan Zakharchuk', '01/27/88', 'Mediterranean', 'Kiev', 10),
 ('Anjanette Vanessa Jia Craynock Santiago Delgado Betancourt Rivera',
  '10/03/87',
  'Japanese',
  'Carolina, Puerto Rico',
  5),
 ('Anjanette Vanessa Jia Craynock Santiago Delgado Betancourt Rivera',
  '10/03/87',
  'Burgers',
  'Carolina, Puerto Rico',
  5),
 ('Anjanette Vanessa Jia Craynock Santiago Delgado Betancourt Rivera',
  '10/03/87',
  'Authentic Bacalaitos',
  'Carolina, Puerto Rico',
  5),
 ('Guy Monahan', '07/21/88', 'Dumplings', 'Jersey City, NJ', 31),
 ('Jonathan Silverman', '08/04/82', 'pizza', 'Brooklyn, NY', 37)]

Now that we have this list of tuples, lets use `executemany()` to insert the data.

In [8]:
stmt = """INSERT INTO students (name, birthday, 
fav_food, birthplace, years_in_nyc)
VALUES (?, ?, ?, ?, ?)
"""

Now that we have the data in the database, we can write

In [9]:
cur.executemany(stmt, students_list)

***You do not need to worry about commiting your changes in a SQLite DB***

### Query your database to see if you have the data

In [10]:
cur.execute("SELECT * FROM students")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,name,birthday,fav_food,birthplace,years_in_nyc
0,Sean Abu Wilson,02/06/1985,Bread,"Birmingham, AL",9
1,christa,01/11/1996,buffalo wings,new york,2
2,Dariga Kokenova,04/19/1987,Italian,Kazakhstan,14
3,Rafael Ferreira,10/13/1991,Linguine con Vongole,"Denver, CO",5
4,Saad Raees,05-13-1992,ice cream,"Karachi,Pakistan",10
5,Ivan Zakharchuk,01.27.1988,Mediterranean,Kiev,10
6,Anjanette Vanessa Jia Craynock Santiago Delgad...,10-03-1987,Japanese,"Carolina, Puerto Rico",5
7,Anjanette Vanessa Jia Craynock Santiago Delgad...,10-03-1987,Burgers,"Carolina, Puerto Rico",5
8,Anjanette Vanessa Jia Craynock Santiago Delgad...,10-03-1987,Authentic Bacalaitos,"Carolina, Puerto Rico",5
9,Guy Monahan,07/21/1988,Dumplings,"Jersey City, NJ",31


### Write queries to answer the following questions:

**Questions**
- Which student was born closest to the cohort's graduation date?
- Which student has the most siblings?
- How many students are only children?
- Which 3 students have lived in NYC the shortest amount of time?
- How many students are native New Yorkers?
- Do any two students have the same favorite food?

In [11]:
cur.execute("""
SELECT * FROM students ORDER BY years_in_nyc ASC LIMIT 3;
""").fetchall()


[('christa', '01/11/1996', 'buffalo wings', 'new york', 2),
 ('Rafael Ferreira', '10/13/1991', 'Linguine con Vongole', 'Denver, CO', 5),
 ('Anjanette Vanessa Jia Craynock Santiago Delgado Betancourt Rivera',
  '10-03-1987',
  'Japanese',
  'Carolina, Puerto Rico',
  5)]

In [18]:
cur.execute("""
SELECT count(name) 
FROM students 
WHERE birthplace LIKE '%NY' 
OR 
birthplace LIKE '%new york%';
""").fetchall()

# % percent sign or modulous to capture like 

[(2,)]

In [17]:
cur.execute("""
SELECT name, count(fav_food) 
FROM students 
GROUP BY fav_food 
HAVING COUNT (fav_food) > 1
""").fetchall()

[]

In [ ]:
cur.execute("""
SELECT name, birthday FROM students WHERE 
""")